In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-09-23"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
342,2024-09-23,Filipinas Copa Dos Governadores,06:00,Magnolia Hotshots,Converge FiberXers,1.39,2.89,198.5,1.87,1.79,-6.5,1.91,1.80,bwi8elJ7,0.719424,0.346021,0.534759,0.558659,0.065445,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.00,148.80,0.00,0.0,0.0,0.0,0.0,0.495636,0.030912,0.041931,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.0,0.0,0.0
343,2024-09-23,Filipinas Copa Dos Governadores,08:30,Rain Or Shine Paint.,Blackwater Bossing,1.35,3.09,223.5,1.85,1.85,-7.5,1.95,1.76,vZTfJiBE,0.740741,0.323625,0.540541,0.540541,0.064365,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.00,121.80,421.80,0.0,1.0,0.0,0.0,0.554219,0.000000,0.072426,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.0,0.0,0.0
344,2024-09-23,Mundo Amistoso Interclubes,03:00,Ryukyu (Jpn),LG Sakers (Kor),1.18,4.47,160.5,1.85,1.85,-11.5,1.95,1.73,ID2J731E,0.847458,0.223714,0.540541,0.540541,0.071171,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.823498,0.000000,0.084545,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.0,0.0,0.0
345,2024-09-23,Paraguai Lnb,20:30,Felix Perez Cardozo,Ciudad Nueva,1.02,8.80,144.5,1.83,1.83,-19.5,1.95,1.72,jsTNSY55,0.980392,0.113636,0.546448,0.546448,0.094029,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.00,163.40,0.00,0.0,0.0,0.0,0.0,1.120426,0.000000,0.088629,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.0,0.0,0.0
346,2024-09-23,Paraguai Lnb,20:30,Olimpia,Dep. San Jose,2.01,1.68,147.5,1.85,1.85,1.5,1.88,1.78,rcsXOj5U,0.497512,0.595238,0.540541,0.540541,0.092751,0.0,0.0,NaN,NaN,245.272,94.092745,0.383626,NaN,NaN,NaN,0.000,0.00,202.73,146.16,0.0,0.0,0.0,0.0,0.126474,0.000000,0.038640,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.0,0.0,0.0
347,2024-09-23,Romênia Divisão A,13:00,CSU Sibiu,Brasov,5.72,1.12,162.5,1.83,1.83,7.5,2.00,1.69,xpMkr0BL,0.174825,0.892857,0.546448,0.546448,0.067682,0.6,0.6,NaN,NaN,220.932,169.972190,0.769342,351.792,267.860008,0.761416,160.964,431.45,142.83,437.61,0.0,1.0,0.0,0.0,0.951079,0.000000,0.118809,1.23,0.246,19.186992,0.561774,0.5,-0.061774,-3.03,-0.606,-0.19802,0.0,0.0,0.0
348,2024-09-23,Turquia Tbl,13:00,Bordo Sportif,Balikesir,1.61,2.12,151.5,1.84,1.82,-3.5,1.90,1.76,zX3oV2eC,0.621118,0.471698,0.543478,0.549451,0.092816,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.193364,0.007728,0.054096,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.0,0.0,0.0
349,2024-09-23,Israel Copa Da Liga,14:00,Hapoel Holon,Nes Ziona,1.19,3.93,161.5,1.82,1.84,-9.5,1.97,1.71,Oh2NnUdQ,0.840336,0.254453,0.549451,0.543478,0.094789,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.756825,0.007728,0.099917,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos
